In [1]:
import os
import sys
initial_path = set(sys.path)
CARLA_EGG_PATH = "/home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg"
sys.path.append(CARLA_EGG_PATH)

# ADD 
try:
    sys.path.append(os.path.abspath('.') + '/PythonAPI/carla')
except IndexError:
    pass

new_paths = set(sys.path) - initial_path
for path in new_paths:
    print(f"Added: {path} to the Path")

Added: /home/rudy/Documents/carla-dataset-runner/playground/PythonAPI/carla to the Path
Added: /home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg to the Path


In [2]:
import carla
from agents.navigation.behavior_agent import BehaviorAgent
import logging

In [3]:
class NPCClass:
    def __init__(self):
        logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
        self.vehicles_list = []
        self.walkers_list = []
        self.all_id = []
        self.client = carla.Client('127.0.0.1', 2000)
        self.client.set_timeout(20.0)

    def create_npcs(self, number_of_vehicles=150, number_of_walkers=70):
        world = self.client.get_world()
        blueprints = world.get_blueprint_library().filter('vehicle.*')
        blueprints = [x for x in blueprints if not x.id.endswith('etron')]  # This guy has a wonky movement [physics broken?]
        blueprintsWalkers = world.get_blueprint_library().filter('walker.pedestrian.*')

        spawn_points = world.get_map().get_spawn_points()
        number_of_spawn_points = len(spawn_points)

        if number_of_vehicles < number_of_spawn_points:
            random.shuffle(spawn_points)
        elif number_of_vehicles > number_of_spawn_points:
            msg = 'requested %d vehicles, but could only find %d spawn points'
            logging.warning(msg, number_of_vehicles, number_of_spawn_points)
            number_of_vehicles = number_of_spawn_points

        # @todo cannot import these directly.
        SpawnActor = carla.command.SpawnActor
        SetAutopilot = carla.command.SetAutopilot
        FutureActor = carla.command.FutureActor

        # --------------
        # Spawn vehicles
        # --------------
        batch = []
        for n, transform in enumerate(spawn_points):
            if n >= number_of_vehicles:
                break
            blueprint = random.choice(blueprints)
            # Taking out bicycles and motorcycles, since the semantic/bb labeling for that is mixed with pedestrian
            if int(blueprint.get_attribute('number_of_wheels')) > 2:
                if blueprint.has_attribute('color'):
                    color = random.choice(blueprint.get_attribute('color').recommended_values)
                    blueprint.set_attribute('color', color)
                if blueprint.has_attribute('driver_id'):
                    driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
                    blueprint.set_attribute('driver_id', driver_id)
                blueprint.set_attribute('role_name', 'autopilot')
                batch.append(SpawnActor(blueprint, transform).then(SetAutopilot(FutureActor, True)))

        for response in self.client.apply_batch_sync(batch):
            if response.error:
                logging.error(response.error)
            else:
                self.vehicles_list.append(response.actor_id)

        # -------------
        # Spawn Walkers
        # -------------
        # 1. take all the random locations to spawn
        spawn_points = []
        for i in range(number_of_walkers):
            spawn_point = carla.Transform()
            loc = world.get_random_location_from_navigation()
            if (loc != None):
                spawn_point.location = loc
                spawn_points.append(spawn_point)

        # 2. we spawn the walker object
        batch = []
        for spawn_point in spawn_points:
            walker_bp = random.choice(blueprintsWalkers)
            # set as not invencible
            if walker_bp.has_attribute('is_invincible'):
                walker_bp.set_attribute('is_invincible', 'false')
            batch.append(SpawnActor(walker_bp, spawn_point))
        results = self.client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                self.walkers_list.append({"id": results[i].actor_id})

        # 3. we spawn the walker controller
        batch = []
        walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
        for i in range(len(self.walkers_list)):
            batch.append(SpawnActor(walker_controller_bp, carla.Transform(), self.walkers_list[i]["id"]))
        results = self.client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                self.walkers_list[i]["con"] = results[i].actor_id
        # 4. we put altogether the walkers and controllers id to get the objects from their id
        for i in range(len(self.walkers_list)):
            self.all_id.append(self.walkers_list[i]["con"])
            self.all_id.append(self.walkers_list[i]["id"])
        self.all_actors = world.get_actors(self.all_id)

        # wait for a tick to ensure client receives the last transform of the walkers we have just created
        world.tick()
        world.wait_for_tick()

        # 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
        for i in range(0, len(self.all_id), 2):
            # start walker
            self.all_actors[i].start()
            # set walk to random point
            self.all_actors[i].go_to_location(world.get_random_location_from_navigation())
            # random max speed
            self.all_actors[i].set_max_speed(1 + random.random()/2)    # max speed between 1 and 2 (default is 1.4 m/s)
        print('Spawned %d vehicles and %d walkers' % (len(self.vehicles_list), len(self.walkers_list)))
        return self.vehicles_list, self.walkers_list

    def remove_npcs(self):
        print('Destroying %d NPC vehicles' % len(self.vehicles_list))
        self.client.apply_batch([carla.command.DestroyActor(x) for x in self.vehicles_list])

        # stop walker controllers (list is [controler, actor, controller, actor ...])
        for i in range(0, len(self.all_id), 2):
            self.all_actors[i].stop()

        print('Destroying %d NPC walkers' % len(self.walkers_list))
        self.client.apply_batch([carla.command.DestroyActor(x) for x in self.all_id])


In [4]:
try:
    import queue
except ImportError:
    import Queue as queue


class CarlaSyncMode(object):
    """
    Context manager to synchronize output from different sensors. Synchronous
    mode is enabled as long as we are inside this context

        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, *sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 20)
        self._queues = []
        self._settings = None

    def __enter__(self):
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout):
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        self.world.apply_settings(self._settings)

    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data

    def tick_no_data(self):
        self.frame = self.world.tick()

In [5]:
import random
import time
import numpy as np
import cv2


class CarlaWorld:
    def __init__(self, world='Town02'):
        
        # Carla initialization
        client = carla.Client('localhost', 2000)
        client.set_timeout(20.0)
        client.load_world(world)
        self.world = client.get_world()
        print('Successfully connected to CARLA')
        self.map = self.world.get_map()
        self.blueprint_library = self.world.get_blueprint_library()
        # Sensors stuff
        self.camera_x_location = 1.0
        self.camera_y_location = 0.0
        self.camera_z_location = 2.0
        self.sensors_list = []
        
        self.total_recorded_frames = 0
        self.first_time_simulating = True

    def remove_npcs(self):
        print('Destroying actors...')
        self.NPC.remove_npcs()
        print('Done destroying actors.')

    def spawn_npcs(self, number_of_vehicles, number_of_walkers):
        self.NPC = NPCClass()
        self.vehicles_list, _ = self.NPC.create_npcs(number_of_vehicles, number_of_walkers)

    def put_rgb_sensor(self, vehicle, sensor_width=640, sensor_height=480, fov=110):
        # https://carla.readthedocs.io/en/latest/cameras_and_sensors/
        bp = self.blueprint_library.find('sensor.camera.rgb')
        # bp.set_attribute('enable_postprocess_effects', 'True')  # https://carla.readthedocs.io/en/latest/bp_library/
        bp.set_attribute('image_size_x', f'{sensor_width}')
        bp.set_attribute('image_size_y', f'{sensor_height}')
        bp.set_attribute('fov', f'{fov}')

        # Adjust sensor relative position to the vehicle
        spawn_point = carla.Transform(carla.Location(x=self.camera_x_location, z=self.camera_z_location))
        self.rgb_camera = self.world.spawn_actor(bp, spawn_point, attach_to=vehicle)
        self.rgb_camera.blur_amount = 0.0
        self.rgb_camera.motion_blur_intensity = 0
        self.rgb_camera.motion_max_distortion = 0

        # Camera calibration
        calibration = np.identity(3)
        calibration[0, 2] = sensor_width / 2.0
        calibration[1, 2] = sensor_height / 2.0
        calibration[0, 0] = calibration[1, 1] = sensor_width / (2.0 * np.tan(fov * np.pi / 360.0))
        self.rgb_camera.calibration = calibration  # Parameter K of the camera
        self.sensors_list.append(self.rgb_camera)
        return self.rgb_camera

    @staticmethod
    def process_rgb_img(img, sensor_width, sensor_height):
        img = np.array(img.raw_data)
        img = img.reshape((sensor_height, sensor_width, 4))
        img = img[:, :, :3]  # taking out opacity channel
        return img

    def remove_sensors(self):
        for sensor in self.sensors_list:
            sensor.destroy()
        self.sensors_list = []
        
    def set_ego_agent(self, vehicle):
        
        ego_agent = BehaviorAgent(vehicle)
        spawn_points = self.map.get_spawn_points()
        random.shuffle(spawn_points)
        if spawn_points[0].location != ego_agent.vehicle.get_location():
            destination = spawn_points[0].location
        else:
            destination = spawn_points[1].location
        ego_agent.set_destination(ego_agent.vehicle.get_location(), destination, clean=True)
        ego_agent.update_information()
        print("Behavior Agent has been set up successfully")
        return ego_agent

    def begin_data_acquisition(self, sensor_width, sensor_height, fov, frames_to_record_one_ego=300, timestamps=[],
                               egos_to_run=1):
        print("Beginning new data acquisition")
        # Changes the ego vehicle to be put the sensor
        current_ego_recorded_frames = 0
        # These vehicles are not considered because the cameras get occluded without changing their absolute position
        ego_vehicle = random.choice([x for x in self.world.get_actors().filter("vehicle.*") if x.type_id not in
                                     ['vehicle.audi.tt', 'vehicle.carlamotors.carlacola', 'vehicle.volkswagen.t2']])
        
        ego_agent = self.set_ego_agent(ego_vehicle)
        
        
        print(f"Using ego: {ego_vehicle.type_id}")
        self.put_rgb_sensor(ego_vehicle, sensor_width, sensor_height, fov)

        # Begin applying the sync mode
        with CarlaSyncMode(self.world, self.rgb_camera, fps=30) as sync_mode:
            # Skip initial frames where the car is being put on the ambient
            if self.first_time_simulating:
                for _ in range(30):
                    sync_mode.tick_no_data()

            while True:
                if current_ego_recorded_frames == frames_to_record_one_ego:
                    print(f"\nRecorded {current_ego_recorded_frames} frames for actual ego")
                    print('------------\n')
                    self.remove_sensors()
                    return timestamps
                
                # Advance the simulation and wait for the data
                # Skip every nth frame for data recording, so that one frame is not that similar to another
                wait_frame_ticks = 0
                while wait_frame_ticks < 1:
                    sync_mode.tick_no_data()
                    
                    wait_frame_ticks += 1
                    ego_agent.run_step()
                    ego_agent.update_information()

                # get data from sensors
                _, rgb_data = sync_mode.tick(timeout=2.0)

                # Processing raw data
                rgb_array = self.process_rgb_img(rgb_data, sensor_width, sensor_height)
                
                cv2.imshow('rgb', rgb_array)
                cv2.waitKey(10)

                ego_wp = self.map.get_waypoint(ego_vehicle.get_location())
                timestamp = round(time.time() * 1000.0)
                
                ego_info = ego_agent.run_step_with_info()
                ego_vehicle.apply_control(ego_info["control"])
                ego_agent.update_information()
    
                current_ego_recorded_frames += 1
                self.total_recorded_frames += 1
                timestamps.append(timestamp)

                sys.stdout.write("\r")
                if ego_info["at_tl"]:
                    sys.stdout.write(f'Frame {self.total_recorded_frames} | {ego_info["command"]} | Light state: {ego_info["tl_state"]} | Distance to TL: {ego_info["tl_distance"]} | Speed: {ego_info["speed"]:.2f} | Distance to center: {ego_info["lane_distance"]} | Orientation: {ego_info["lane_orientation"]}')
                else:
                    sys.stdout.write(f'Frame {self.total_recorded_frames} | {ego_info["command"]} | Speed: {ego_info["speed"]:.2f} | Distance to center: {ego_info["lane_distance"]} | Orientation: {ego_info["lane_orientation"]}')
                sys.stdout.flush()



In [6]:
sensor_width = 800
sensor_height = 600
fov = 110

In [8]:
world = CarlaWorld()
world.spawn_npcs(10, 0)
world.begin_data_acquisition(sensor_width, sensor_height, fov, frames_to_record_one_ego=1000);

Successfully connected to CARLA
Spawned 10 vehicles and 0 walkers
Beginning new data acquisition
Behavior Agent has been set up successfully
Using ego: vehicle.mini.cooperst
Frame 1000 | RoadOption.RIGHT | Speed: 18.39 | Distance to center: 0.6678774623857251 | Orientation: 0.80009928236106636377904.06245423108885362 | Orientation: 0.055198906861524965745
Recorded 1000 frames for actual ego
------------

